In [8]:
"""class AnimalData:
    def __init__(self,root,transforms):
        self.root=root
        self.transforms=transforms
        classes=sort(list(os.listdir(root)))
        images=[]
        for c in classes:
            if c=='Label':
                continue
            for img in sort(list(os.listdir(c))):
                images.append(root+c+img)
        self.images=images
        label_dir=[c+'/Label' for c in classes]
        labels=[]
        for c in classes:
            for label in sort(list(os.listdir(label_dir))):
                labels.append(root+label_dir+label)
        self.labels=labels
    def __getitem__(self,idx):
        img =Image.open(self.img[idx]).convert("RGB")
        with open(self.labels,'r') as f:
            content= f.read()
            lines=contents.split(' ')
            for i in range(1, len(lines)):
                lines[i]=int(lines[i])
        
        """
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import tqdm 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import random
import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
train_labels0=[]
test_labels0=[]
train_imgs0=[]
test_imgs0=[]
classes=['Bear', 'Bull', 'Cat', 'Cattle',
           'Cheetah', 'Chicken', 'Crocodile',
           'Deer', 'Dog', 'Fox', 'Goat', 'Hippopotamus', 
           'Horse', 'Jaguar', 'Leopard', 'Lion', 'Lynx',
           'Monkey', 'Mule', 'Ostrich', 'Owl', 
           'Pig', 'Raccoon', 'Raven', 'Rhinoceros', 
           'Sheep', 'Snake', 'Tiger', 'Turkey', 'Turtle', 'Zebra']
"""classes=['Cat','Dog']"""
encode={}
for i in range(len(classes)):
    encode[classes[i]]=i+1

def checkclass(dirname):
    
    for animal in classes:
        if dirname.endswith(animal):
            return True
    return False

for dirname, dirs, filenames in os.walk('./data/'):
    
    if 'train' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        
        for label in sorted(os.listdir(labels)):
            
            
            if label.endswith('txt'):
                train_labels0.append(labels+label)
        
        for image in sorted(filenames):
            
            
            if image.endswith('png') or image.endswith('jpg'):
                train_imgs0.append(dirname+'/'+image)
    if 'val' in dirname and checkclass(dirname):
        labels= dirname +'/Label/'
        for label in sorted(os.listdir(labels)):
            if label.endswith('.txt'):
                test_labels0.append(labels+label)
        for image in sorted(filenames):
            if image.endswith('png')or image.endswith('jpg'):
                test_imgs0.append(dirname+'/'+image)


In [10]:
print(len(train_imgs0))
print(len(train_labels0))
print(len(test_imgs0))
print(len(test_labels0))
train_labels=[]
test_labels=[]
train_imgs=[]
test_imgs=[]
for i in range(len(train_labels0)):
    if i%10==0:
        
        train_labels.append(train_labels0[i])
        train_imgs.append(train_imgs0[i])
for i in range(len(test_labels0)):
    if i%20==0:
        test_labels.append(test_labels0[i])
        test_imgs.append(test_imgs0[i])
print(len(train_imgs))
print(len(train_labels))
print(len(test_imgs))
print(len(test_labels))


14620
14620
6864
6864
1462
1462
344
344


In [11]:
class AnimalDataset(object):
    def __init__(self, transforms,img_dir,label_dir):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = img_dir
        self.label=label_dir
#         self.labels = list(sorted(os.listdir("/kaggle/input/face-mask-detection/annotations/")))

    def __getitem__(self, idx):
        # load images ad masks
        file_image = self.imgs[idx]
        file_label = self.label[idx]
        img = Image.open(file_image).convert("RGB")
        target={}
        target['image_id']=torch.tensor([idx])
        labels=[]
        boxes=[]
        #Generate Label
        with open(file_label,'r')as f:
            lines=f.readlines()
            for line in lines:
                line_list=line.strip().split(' ')
                labels.append(encode[line_list[0]])
                boxes.append([float(i) for i in line_list[1:]])
                
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(labels, dtype=torch.int64)        
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        
        iscrowd = torch.zeros((len(classes),), dtype=torch.int64)

        target['boxes']=boxes
        target['labels']=labels
       
        target["area"] = area
        target["iscrowd"] = iscrowd
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [12]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [13]:
def get_model_instance_mobilenet(num_classes):
    
    backbone = torchvision.models.mobilenet_v2(pretrained=True).features
    # FasterRCNN needs to know the number of
    # output channels in a backbone. For mobilenet_v2, it's 1280
    # so we need to add it here
    backbone.out_channels = 1280

    # let's make the RPN generate 5 x 3 anchors per spatial
    # location, with 5 different sizes and 3 different aspect
    # ratios. We have a Tuple[Tuple[int]] because each feature
    # map could potentially have different sizes and
    # aspect ratios 
    anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                       aspect_ratios=((0.5, 1.0, 2.0),))

    # let's define what are the feature maps that we will
    # use to perform the region of interest cropping, as well as
    # the size of the crop after rescaling.
    # if your backbone returns a Tensor, featmap_names is expected to
    # be [0]. More generally, the backbone should return an
    # OrderedDict[Tensor], and in featmap_names you can choose which
    # feature maps to use.
    roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                    output_size=7,
                                                    sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(backbone,
                       num_classes=2,
                       rpn_anchor_generator=anchor_generator,
                       box_roi_pool=roi_pooler)
    return model


In [14]:

from engine import train_one_epoch , evaluate
import utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = len(classes)+1
    # use our dataset and defined transformations
    train_data=AnimalDataset(get_transform(train=True),train_imgs,train_labels)
    test_data=AnimalDataset(get_transform(train=False),test_imgs,test_labels)

    # define training and validation data loaders
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=4, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_mobilenet(num_classes)

    # move model to the right device
    model.to(device)
    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        torch.save(model.state_dict(), "./model/fasterRCNN/fasterRCNN_mobilenet"+str(epoch)+".pt")
        # evaluate on the test dataset
        evaluate(model, train_loader, device=device)

    print("That's it!")

In [12]:
from engine import train_one_epoch, evaluate
import utils
test_data=AnimalDataset(data_transform,test_imgs[:10],test_labels[:10])
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person


# define training and validation data loaders
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

# get the model using our helper function


# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size=3,
                                                gamma=0.1)

# let's train it for 10 epochs
num_epochs = 10

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model,test_loader, device=device)


0it [00:00, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.56 GiB (GPU 0; 8.00 GiB total capacity; 6.19 GiB already allocated; 0 bytes free; 6.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [42]:

test_data=AnimalDataset(data_transform,test_imgs[:10],test_labels[:10])

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)
evaluate(model,test_loader, device=device)

creating index...
index created!


In [11]:
torch.cuda.empty_cache()

In [ ]:
ssdmodel=